In [ ]:
pip install -q datasets


     |████████████████████████████████| 325 kB 10.1 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 134 kB 12.4 MB/s 
     |████████████████████████████████| 1.1 MB 54.9 MB/s 
     |████████████████████████████████| 212 kB 60.8 MB/s 
     |████████████████████████████████| 127 kB 66.2 MB/s 
     |████████████████████████████████| 271 kB 70.7 MB/s 
     |████████████████████████████████| 144 kB 71.0 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
pip install -q transformers

     |████████████████████████████████| 3.8 MB 8.3 MB/s 
     |████████████████████████████████| 6.5 MB 45.2 MB/s 
     |████████████████████████████████| 895 kB 61.4 MB/s 
     |████████████████████████████████| 596 kB 61.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.figure_factory as ff
from transformers import TFAutoModel
from transformers import TrainingArguments


In [ ]:
# tTrain = pd.read_csv("NLP_train.csv")
# tTrain.head()

In [ ]:
# tTrain = tTrain[['fixed_text', 'keywords_label']]
# tTrain.head()

In [ ]:
# bertweet = TFAutoModel.from_pretrained("vinai/bertweet-base")

In [ ]:
# bertweet.summary()

In [ ]:
# https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'NLP_train.csv', 'test': 'NLP_test.csv'}, encoding = "ISO-8859-1")


Using custom data configuration default-2ff88fdcedef4f2b


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2ff88fdcedef4f2b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
type(dataset)
dataset['train'][0]

{'fixed_text': 'cross eyed and tired long day lymedisease',
 'id': 579785953885040641,
 'keywords_label': 1,
 'text': 'Cross eyed and tired. Long day. #LymeDisease https://t.co/lieufFyLxq'}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'keywords_label', 'fixed_text'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['id', 'text', 'keywords_label', 'fixed_text'],
        num_rows: 7500
    })
})

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# # For transformers v3.x:
# # tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

# # INPUT TWEET IS ALREADY NORMALIZED!
# line = "SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL via @USER :cry:"

# input_ids = torch.tensor([tokenizer.encode(line)])

# with torch.no_grad():
#     features = bertweet(input_ids)  # Models outputs are now tuples

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")



emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer("hello my world")

{'input_ids': [0, 2279, 23, 220, 2], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
def transform_labels(label):

    num = label['keywords_label']
    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['fixed_text'], padding='max_length')

dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['id','text','keywords_label','fixed_text']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/70000 [00:00<?, ?ex/s]

  0%|          | 0/7500 [00:00<?, ?ex/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7500
    })
})

In [ ]:
# train_dataset = dataset['train'].shuffle(seed=10).select(range(3000))
# eval_dataset = dataset['train'].shuffle(seed=10).select(range(3000, 3500))

In [ ]:
# from transformers import Trainer
# from transformers import TrainingArguments

# training_args = TrainingArguments("test_trainer", num_train_epochs=1)


# trainer = Trainer(
#     model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
# )
# trainer.train()


In [ ]:
# from datasets import load_metric

# metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

# trainer.evaluate()

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base")


loading configuration file https://huggingface.co/vinai/bertweet-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/356366feedcea0917e30f7f235e1e062ffc2d28138445d5672a184be756c8686.a2b6026e688d1b19cebc0981d8f3a5b1668eabfda55b2c42049d5eac0bc8cb2d
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertweetTokenizer",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache"

In [ ]:

# bertweet = AutoModel.from_pretrained("vinai/bertweet-large")
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

train_dataset = dataset['train'].shuffle(seed=10)
eval_dataset = dataset['test'].shuffle(seed=10)

# train_dataset = dataset['train'].shuffle(seed=10).select(range(3000))
# eval_dataset = dataset['train'].shuffle(seed=10).select(range(3000, 3500))


training_args = TrainingArguments("test_trainer", num_train_epochs=1)




Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-2ff88fdcedef4f2b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-01a8df755bacedf8.arrow
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 70000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8750


Step,Training Loss
500,0.285400
1000,0.286100
1500,0.237400
2000,0.193800
2500,0.178300
3000,0.186200
3500,0.178700
4000,0.178100
4500,0.153900
5000,0.161900


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=8750, training_loss=0.1693055938720703, metrics={'train_runtime': 2055.5265, 'train_samples_per_second': 34.055, 'train_steps_per_second': 4.257, 'total_flos': 4604443468800000.0, 'train_loss': 0.1693055938720703, 'epoch': 1.0})

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer.evaluate()

***** Running Evaluation *****
  Num examples = 7500
  Batch size = 8


{'epoch': 1.0,
 'eval_loss': 0.4974231719970703,
 'eval_runtime': 58.4043,
 'eval_samples_per_second': 128.415,
 'eval_steps_per_second': 16.06}

In [ ]:
metric

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    normalize: If False, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight: Sample weights.
Returns:
    accuracy: Accuracy score.
Examples:

    >>> accuracy_metric = datasets.load_metric("accuracy")
    >>> results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
    >>> print(results)
    {'accuracy': 1.0}
""", stored examples: 0)

In [ ]:
accuracy_metric = load_metric("accuracy")
results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
print(results)

{'accuracy': 1.0}
